In [1]:
import os
import cv2
import time
import numpy as np
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.applications.resnet50 import preprocess_input
from keras.applications import *
from keras import regularizers

In [2]:
v_path = r"F:\violent_video_tagging\violence-video-classification\data\raw_frames\train\violent\\"
nv_path = r"F:\violent_video_tagging\violence-video-classification\data\raw_frames\train\non_violent\\"

# v_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\violent\\"
# nv_path = r"F:\violent_video_tagging\violence-video-classification\data\final\train\non_violent\\"

In [3]:
violent_frames = []
non_violent_frames = []

for frame in os.listdir(v_path):
    frame = cv2.imread(os.path.join(v_path,frame))
    violent_frames.append(frame)


for frame in os.listdir(nv_path):
    frame = cv2.imread(os.path.join(nv_path,frame))
    non_violent_frames.append(frame)

In [32]:
violent_frames[0].shape

(224, 224, 3)

In [33]:
violent_frames[0].shape

(224, 224, 3)

In [6]:
violent_frames.shape

AttributeError: 'list' object has no attribute 'shape'

In [4]:
#Creates sequence 
# SHAPE : (N_SAMPLES,TIMESTEP,WIDTH,HEIGHT,CHANNEL)
def create_seq(violent_frames, non_violent_frames):
    
    print("+++ Creating Sequence... +++")
    
    violent_vid = []
    non_violent_vid = []

    i = 0
    while i < len(violent_frames):
        violent_vid.append(violent_frames[i:i+30])
        i = i+30


    i = 0
    while i < len(non_violent_frames):
        non_violent_vid.append(non_violent_frames[i:i+30])
        i = i+30
        
    violent_vid = np.asarray(violent_vid)
    non_violent_vid = np.asarray(non_violent_vid)
    
    return violent_vid, non_violent_vid


In [5]:
def result(x,y):
    
    print("+++ Generating result... +++")
    
    pred = model.predict(x)
#     print('First prediction:', pred)
    
    score = model.evaluate(x, y,verbose=1)
    print("-----------------------------")
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    prediction = []
    for p in pred:
        if p>=.5:
            prediction.append(1)
        else:
            prediction.append(0)
    print("-----------------------------")
    print("Classification report")
    print("-----------------------------")
    print(classification_report(y, prediction))
    
    print("-----------------------------")
    print("Confusion Matrix")
    print("-----------------------------")
    conf_mat = confusion_matrix(y, prediction)
    print(conf_mat)
    

In [6]:
def preprocess_lstm(features):
    
    print("+++ Preprocessing data... +++")
    
    violent_features = features[0:30*math.floor(violent_frames.shape[0]/30)]
    non_violent_features = features[violent_frames.shape[0]:violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)]


    print("Violent features: ", violent_features.shape)
    print("Non Violent features: ", non_violent_features.shape)


    violent_vid,non_violent_vid = create_seq(violent_features, non_violent_features)
    violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

    print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
    print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)


    test_x = np.vstack((violent_vid,non_violent_vid))
    test_y = np.append(violent_y, non_violent_y)

    print("Total data: ", test_x.shape)
    print("Total target: ", test_y.shape)

    test_x = np.reshape(test_x, (test_x.shape[0],test_x.shape[1],np.prod(test_x.shape[2:])))
    print("(LSTM) After Rehshape: ", test_x.shape)
    
    return test_x,test_y

In [7]:
def feature_extract(violent_frames, non_violent_frames, model):
    
    print("+++ Extracting feature... +++")
    
    violent_frames = np.asarray(violent_frames)
    non_violent_frames = np.asarray(non_violent_frames)

    print ("Before Feature extraction: ")
    print(violent_frames.shape,non_violent_frames.shape)
    all_data = np.vstack((violent_frames,non_violent_frames))
    print("Adding all data: ", all_data.shape)

    #creates feature descriptors
    all_data = all_data.astype('float64')
    desc = preprocess_input(all_data)
    if(model == 'resnet50'):
        loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg19'):
        loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg16'):
        loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
    else:
        print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")
        
    loaded_model = Model(loaded_model.input,loaded_model.output)
    features = loaded_model.predict(desc,batch_size=10,verbose=1)

    print ("After Feature extraction: ", features.shape)
    
    return features

In [53]:
non_violent_frames.shape

(11520, 7, 7, 2048)

In [12]:
print("+++ Extracting feature... +++")

violent_frames = np.asarray(violent_frames)
non_violent_frames = np.asarray(non_violent_frames)

print ("Before Feature extraction: ")
print(violent_frames.shape,non_violent_frames.shape)
all_data = np.vstack((violent_frames,non_violent_frames))
print("Adding all data: ", all_data.shape)

#creates feature descriptors
all_data = all_data.astype('float64')
desc = preprocess_input(all_data)
model = 'resnet50'
if(model == 'resnet50'):
    loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg19'):
    loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
elif(model == 'vgg16'):
    loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
else:
    print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")

+++ Extracting feature... +++
Before Feature extraction: 
(27853, 224, 224, 3) (25404, 224, 224, 3)
Adding all data:  (53257, 224, 224, 3)


In [56]:
features.shape

(31183, 7, 7, 2048)

In [11]:
features = np.load("resnet_features.npy")

In [13]:
 print ("After Feature extraction: ", features.shape)

After Feature extraction:  (26863, 7, 7, 2048)


In [87]:

loaded_model = Model(loaded_model.input,loaded_model.output)
features = loaded_model.predict(desc,batch_size=10,verbose=1)

print ("After Feature extraction: ", features.shape)

2687/2687 [==============================] - 1768s 658ms/step
After Feature extraction:  (26863, 7, 7, 2048)


In [97]:
print("+++ Preprocessing data... +++")

violent_features = features[0:15300]
non_violent_features = features[15323:26843]

print("Violent features: ", violent_features.shape)
print("Non Violent features: ", non_violent_features.shape)

+++ Preprocessing data... +++
Violent features:  (15300, 7, 7, 2048)
Non Violent features:  (11520, 7, 7, 2048)


In [41]:
violent_frames = violent_features
non_violent_frames = non_violent_features

violent_vid = []
non_violent_vid = []

i = 0
while i < len(violent_frames):
    violent_vid.append(violent_frames[i:i+30])
    i = i+30


i = 0
while i < len(non_violent_frames):
    non_violent_vid.append(non_violent_frames[i:i+30])
    i = i+30

violent_vid = np.asarray(violent_vid)
non_violent_vid = np.asarray(non_violent_vid)

In [99]:
violent_vid.shape

(510, 30, 7, 7, 2048)

In [43]:
non_violent_vid.shape

(384, 30, 7, 7, 2048)

In [75]:
features = feature_extract(violent_frames, non_violent_frames, 'resnet50')
test_x, test_y = preprocess_lstm(features)

+++ Extracting feature... +++
Before Feature extraction: 
(240, 7, 7, 2048) (240, 7, 7, 2048)
Adding all data:  (480, 7, 7, 2048)


ValueError: in user code:

    c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1122 pre

In [8]:
features = feature_extract(violent_frames, non_violent_frames, 'resnet50')


+++ Extracting feature... +++
Before Feature extraction: 
(27853, 224, 224, 3) (25404, 224, 224, 3)
Adding all data:  (53257, 224, 224, 3)
5326/5326 [==============================] - 2417s 454ms/step
After Feature extraction:  (53257, 7, 7, 2048)


In [44]:
test_x, test_y = preprocess_lstm(features)

+++ Preprocessing data... +++
Violent features:  (15300, 7, 7, 2048)
Non Violent features:  (11520, 7, 7, 2048)
+++ Creating Sequence... +++
Violent Video Seq:  (510, 30, 7, 7, 2048) Non_violent video Seq:  (384, 30, 7, 7, 2048)
Violent Label:  (510,) Non_violent Label:  (384,)
Total data:  (894, 30, 7, 7, 2048)
Total target:  (894,)
(LSTM) After Rehshape:  (894, 30, 100352)


In [26]:
test_x.shape

(894, 30, 100352)

In [28]:
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM.h5')
model.summary()
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50)                20080600  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________


In [29]:
result(test_x,test_y)

+++ Generating result... +++
28/28 [==============================] - 19s 669ms/step - loss: 0.1060 - accuracy: 0.9978
-----------------------------
Test loss: 0.10598662495613098
Test accuracy: 0.9977628588676453
-----------------------------
Classification report
-----------------------------
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       510
         1.0       0.99      1.00      1.00       384

    accuracy                           1.00       894
   macro avg       1.00      1.00      1.00       894
weighted avg       1.00      1.00      1.00       894

-----------------------------
Confusion Matrix
-----------------------------
[[508   2]
 [  0 384]]


In [10]:
features.shape

(53257, 7, 7, 2048)

In [13]:
non_violent_frames.shape[0]

25404

In [14]:
import math
print("+++ Preprocessing data... +++")

violent_features = features[0:30*math.floor(violent_frames.shape[0]/30)]
non_violent_features = features[violent_frames.shape[0]:violent_frames.shape[0]+30*math.floor(non_violent_frames.shape[0]/30)]


print("Violent features: ", violent_features.shape)
print("Non Violent features: ", non_violent_features.shape)


violent_vid,non_violent_vid = create_seq(violent_features, non_violent_features)
violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)


test_x = np.vstack((violent_vid,non_violent_vid))
test_y = np.append(violent_y, non_violent_y)

print("Total data: ", test_x.shape)
print("Total target: ", test_y.shape)

+++ Preprocessing data... +++
Violent features:  (27840, 7, 7, 2048)
Non Violent features:  (25380, 7, 7, 2048)
+++ Creating Sequence... +++
Violent Video Seq:  (928, 30, 7, 7, 2048) Non_violent video Seq:  (846, 30, 7, 7, 2048)
Violent Label:  (928,) Non_violent Label:  (846,)
Total data:  (1774, 30, 7, 7, 2048)
Total target:  (1774,)


In [15]:
test_x.shape

(1774, 30, 7, 7, 2048)

In [16]:
with open('resnet_features3.npy', 'wb') as f:
    np.save(f, test_x)

In [69]:
print("+++ Preprocessing data... +++")

violent_features = features[0:240]
non_violent_features = features[240:]

print("Violent features: ", violent_features.shape)
print("Non Violent features: ", non_violent_features.shape)




+++ Preprocessing data... +++
Violent features:  (240, 7, 7, 2048)
Non Violent features:  (240, 7, 7, 2048)


In [94]:
violent_frames = violent_features
non_violent_frames = non_violent_features

violent_vid = []
non_violent_vid = []

i = 0
while i < len(violent_frames):
    violent_vid.append(violent_frames[i:i+30])
    i = i+30


i = 0
while i < len(non_violent_frames):
    non_violent_vid.append(non_violent_frames[i:i+30])
    i = i+30

violent_vid = np.asarray(violent_vid)
non_violent_vid = np.asarray(non_violent_vid)

In [95]:
non_violent_vid.shape

(385,)

In [101]:
violent_vid,non_violent_vid = create_seq(violent_features, non_violent_features)
violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)


+++ Creating Sequence... +++
Violent Video Seq:  (510, 30, 7, 7, 2048) Non_violent video Seq:  (384, 30, 7, 7, 2048)
Violent Label:  (510,) Non_violent Label:  (384,)


In [104]:


test_x = np.vstack((violent_vid,non_violent_vid))




In [ ]:
input.shape[0],input.shape[1],input.shape[2]*input.shape[3]

In [105]:
test_x.shape

(894, 30, 7, 7, 2048)

In [107]:
test_x = np.reshape(test_x, (test_x.shape[0],test_x.shape[1],np.prod(test_x.shape[2:])))
print("(LSTM) After Rehshape: ", test_x.shape)

(LSTM) After Rehshape:  (894, 30, 100352)


In [100]:
with open('resnet_features.npy', 'wb') as f:
    np.save(f, features)

In [103]:
from keras import regularizers
model = Sequential()
model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.load_weights('resnet_LSTM.h5')
model.summary()
optimizer = optimizers.adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])


result(test_x,test_y)

ValueError: Shapes (100352, 200) and (14336, 200) are incompatible